In [1]:
import sqlite3
import pandas as pd
import os
from datetime import datetime

In [2]:
# create database directory
os.makedirs('data', exist_ok=True)

# connect to sqlite database
db_path = 'data/analysis.db'
conn = sqlite3.connect(db_path)
print(f"Connected to database: {db_path}")

Connected to database: data/analysis.db


In [3]:
# create air quality table
conn.execute('''
    CREATE TABLE IF NOT EXISTS air_quality (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        date_local TEXT,
        state_code TEXT,
        county_code TEXT,
        site_number TEXT,
        parameter_code TEXT,
        parameter TEXT,
        sample_duration TEXT,
        arithmetic_mean REAL,
        max_value REAL,
        units_of_measure TEXT,
        latitude REAL,
        longitude REAL,
        local_site_name TEXT,
        city TEXT,
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    )
''')

print("Created air_quality table")

Created air_quality table


In [4]:
# create hospital visits table
conn.execute('''
    CREATE TABLE IF NOT EXISTS hospital_visits (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        mmwr_week TEXT,
        week_start TEXT,
        week_end TEXT,
        season TEXT,
        respiratory_category TEXT,
        visit_type TEXT,
        demographic_category TEXT,
        demographic_group TEXT,
        percent REAL,
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    )
''')

print("Created hospital_visits table")

Created hospital_visits table


In [5]:
# create monthly analysis table
conn.execute('''
    CREATE TABLE IF NOT EXISTS monthly_analysis (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        month TEXT,
        respiratory_category TEXT,
        parameter TEXT,
        avg_pollution REAL,
        avg_visit_percent REAL,
        record_count INTEGER,
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
        UNIQUE(month, respiratory_category, parameter)
    )
''')

print("Created monthly_analysis table")

Created monthly_analysis table


In [6]:
# create metadata table
conn.execute('''
    CREATE TABLE IF NOT EXISTS metadata (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        table_name TEXT,
        record_count INTEGER,
        last_updated TIMESTAMP,
        data_source TEXT,
        notes TEXT
    )
''')

conn.commit()
print("Created metadata table")

Created metadata table


In [7]:
# load air quality data from csv
print("Loading air quality data...")
air_df = pd.read_csv('data/air.csv', low_memory=False)

print(f"Loaded {len(air_df):,} raw air quality records")

Loading air quality data...
Loaded 30,697 raw air quality records


In [8]:
# select relevant columns
air_columns = [
    'date_local', 'state_code', 'county_code', 'site_number',
    'parameter_code', 'parameter', 'sample_duration',
    'arithmetic_mean', 'first_max_value', 'units_of_measure',
    'latitude', 'longitude', 'local_site_name', 'city'
]

existing_cols = [col for col in air_columns if col in air_df.columns]
air_clean = air_df[existing_cols].copy()

# rename for consistency
if 'first_max_value' in air_clean.columns:
    air_clean['max_value'] = air_clean['first_max_value']
    air_clean.drop('first_max_value', axis=1, inplace=True)

print(f"Prepared {len(air_clean):,} records with {len(air_clean.columns)} columns")
air_clean.head()

Prepared 30,697 records with 14 columns


,date_local,state_code,county_code,site_number,parameter_code,parameter,sample_duration,arithmetic_mean,units_of_measure,latitude,longitude,local_site_name,city,max_value
0,2015-01-06,17,31,76,88101,PM2.5 - Local Conditions,24 HOUR,10.3,Micrograms/cubic meter (LC),41.7514,-87.713488,COM ED MAINTENANCE BLDG,Chicago,10.3
1,2015-01-06,17,31,76,88101,PM2.5 - Local Conditions,24 HOUR,10.3,Micrograms/cubic meter (LC),41.7514,-87.713488,COM ED MAINTENANCE BLDG,Chicago,10.3
2,2015-01-06,17,31,76,88101,PM2.5 - Local Conditions,24 HOUR,10.3,Micrograms/cubic meter (LC),41.7514,-87.713488,COM ED MAINTENANCE BLDG,Chicago,10.3
3,2015-01-06,17,31,76,88101,PM2.5 - Local Conditions,24 HOUR,10.3,Micrograms/cubic meter (LC),41.7514,-87.713488,COM ED MAINTENANCE BLDG,Chicago,10.3
4,2015-01-06,17,31,76,88101,PM2.5 - Local Conditions,24 HOUR,10.3,Micrograms/cubic meter (LC),41.7514,-87.713488,COM ED MAINTENANCE BLDG,Chicago,10.3


In [9]:
# load into database
air_clean.to_sql('air_quality', conn, if_exists='replace', index=False)
print(f"Loaded {len(air_clean):,} air quality records into database")

# update metadata
conn.execute('''
    INSERT OR REPLACE INTO metadata (table_name, record_count, last_updated, data_source, notes)
    VALUES (?, ?, ?, ?, ?)
''', ('air_quality', len(air_clean), datetime.now().isoformat(), 
      'EPA AQS API (dailyData/byCounty)', 'Chicago PM2.5 daily measurements 2015-2024'))
conn.commit()

Loaded 30,697 air quality records into database


In [10]:
# load hospital visit data from csv
print("Loading hospital visit data...")
hosp_df = pd.read_csv('data/hosp_data.csv')

print(f"Loaded {len(hosp_df):,} hospital visit records")
hosp_df.head()

Loading hospital visit data...


Loaded 55,000 hospital visit records


,Unnamed: 0,mmwr_week,week,week_start,week_end,season,data_source,essence_category,respiratory_category,visit_type,demographic_category,demographic_group,percent,current_week_ending
0,0,202348,48,2023-11-26T00:00:00.000,2023-12-02T00:00:00.000,2023-2024,ESSENCE,CDC COVID-Specific DD v1,COVID-19,ED Visits,Age Group,45_64,0.021902,2025-11-29T00:00:00.000
1,1,202304,4,2023-01-22T00:00:00.000,2023-01-28T00:00:00.000,2022-2023,ESSENCE,CDC Influenza DD v1,Influenza,ED Visits,Age Group,Age Unknown,0.000000,2026-01-31T00:00:00.000
2,2,201749,49,2017-12-03T00:00:00.000,2017-12-09T00:00:00.000,2017-2018,ESSENCE,CDC Respiratory Syncytial Virus DD v1,RSV,ED Visits,Age Group,05_17,0.000000,2025-12-06T00:00:00.000
3,3,201845,45,2018-11-04T00:00:00.000,2018-11-10T00:00:00.000,2018-2019,ESSENCE,CDC Broad Acute Respiratory DD v1,Broad Acute Respiratory,ED Visits,Race/Ethnicity,Unknown Race/Ethnicity,0.125376,2025-11-08T00:00:00.000
4,4,202323,23,2023-06-04T00:00:00.000,2023-06-10T00:00:00.000,2022-2023,ESSENCE,CDC COVID-Specific DD v1,COVID-19,ED Visits,Age Group,00_04,0.007612,2026-06-13T00:00:00.000


In [11]:
# select relevant columns
hosp_columns = [
    'mmwr_week', 'week_start', 'week_end', 'season',
    'respiratory_category', 'visit_type', 'demographic_category',
    'demographic_group', 'percent'
]

existing_hosp_cols = [col for col in hosp_columns if col in hosp_df.columns]
hosp_clean = hosp_df[existing_hosp_cols].copy()

print(f"Prepared {len(hosp_clean):,} records with {len(hosp_clean.columns)} columns")

Prepared 55,000 records with 9 columns


In [12]:
# load into database
hosp_clean.to_sql('hospital_visits', conn, if_exists='replace', index=False)
print(f"Loaded {len(hosp_clean):,} hospital visit records into database")

# update metadata
conn.execute('''
    INSERT OR REPLACE INTO metadata (table_name, record_count, last_updated, data_source, notes)
    VALUES (?, ?, ?, ?, ?)
''', ('hospital_visits', len(hosp_clean), datetime.now().isoformat(),
      'City of Chicago Open Data Portal', 'Weekly respiratory illness ED visits by category and demographics'))
conn.commit()

Loaded 55,000 hospital visit records into database


In [13]:
# create monthly aggregation
print("Creating monthly aggregation...")
air_db = pd.read_sql('SELECT * FROM air_quality', conn)
hosp_db = pd.read_sql('SELECT * FROM hospital_visits', conn)

print(f"Air quality records: {len(air_db):,}")
print(f"Hospital records: {len(hosp_db):,}")

Creating monthly aggregation...


Air quality records: 30,697
Hospital records: 55,000


In [14]:
# convert dates to datetime
air_db['date_local'] = pd.to_datetime(air_db['date_local'], errors='coerce')
hosp_db['week_start'] = pd.to_datetime(hosp_db['week_start'], errors='coerce')

# create month columns
air_db['month'] = air_db['date_local'].dt.to_period('M').astype(str)
hosp_db['month'] = hosp_db['week_start'].dt.to_period('M').astype(str)

print("Converted dates and created month columns")

Converted dates and created month columns


In [15]:
# aggregate air quality by month and parameter
air_monthly = (
    air_db.groupby(['month', 'parameter'])
    .agg({'arithmetic_mean': 'mean'})
    .reset_index()
    .rename(columns={'arithmetic_mean': 'avg_pollution'})
)

print(f"Air quality monthly records: {len(air_monthly):,}")
air_monthly.head()

Air quality monthly records: 120


,month,parameter,avg_pollution
0,2015-01,PM2.5 - Local Conditions,12.624138
1,2015-02,PM2.5 - Local Conditions,12.460714
2,2015-03,PM2.5 - Local Conditions,13.769231
3,2015-04,PM2.5 - Local Conditions,8.935714
4,2015-05,PM2.5 - Local Conditions,9.641935


In [16]:
# aggregate hospital data by month and respiratory category
hosp_monthly = (
    hosp_db.groupby(['month', 'respiratory_category'])
    .agg({'percent': 'mean'})
    .reset_index()
    .rename(columns={'percent': 'avg_visit_percent'})
)

print(f"Hospital monthly records: {len(hosp_monthly):,}")
hosp_monthly.head()

Hospital monthly records: 610


,month,respiratory_category,avg_visit_percent
0,2015-10,Broad Acute Respiratory,0.055516
1,2015-10,COVID-19,0.000000
2,2015-10,ILI,0.022966
3,2015-10,Influenza,0.000645
4,2015-10,RSV,0.000111


In [17]:
# merge datasets on month
merged_monthly = hosp_monthly.merge(air_monthly, on='month', how='inner')
merged_monthly['record_count'] = 1

print(f"Merged monthly records: {len(merged_monthly):,}")
print(f"Date range: {merged_monthly['month'].min()} to {merged_monthly['month'].max()}")
print(f"Respiratory categories: {merged_monthly['respiratory_category'].nunique()}")
print(f"Pollutant parameters: {merged_monthly['parameter'].nunique()}")

merged_monthly.head(10)

Merged monthly records: 555
Date range: 2015-10 to 2024-12
Respiratory categories: 5
Pollutant parameters: 1


,month,respiratory_category,avg_visit_percent,parameter,avg_pollution,record_count
0,2015-10,Broad Acute Respiratory,0.055516,PM2.5 - Local Conditions,7.562857,1
1,2015-10,COVID-19,0.000000,PM2.5 - Local Conditions,7.562857,1
2,2015-10,ILI,0.022966,PM2.5 - Local Conditions,7.562857,1
3,2015-10,Influenza,0.000645,PM2.5 - Local Conditions,7.562857,1
4,2015-10,RSV,0.000111,PM2.5 - Local Conditions,7.562857,1
5,2015-11,Broad Acute Respiratory,0.070276,PM2.5 - Local Conditions,8.206250,1
6,2015-11,COVID-19,0.000000,PM2.5 - Local Conditions,8.206250,1
7,2015-11,ILI,0.023126,PM2.5 - Local Conditions,8.206250,1
8,2015-11,Influenza,0.000758,PM2.5 - Local Conditions,8.206250,1
9,2015-11,RSV,0.000260,PM2.5 - Local Conditions,8.206250,1


In [18]:
# load merged data into database
merged_monthly.to_sql('monthly_analysis', conn, if_exists='replace', index=False)
print(f"Loaded {len(merged_monthly):,} monthly analysis records into database")

# update metadata
conn.execute('''
    INSERT OR REPLACE INTO metadata (table_name, record_count, last_updated, data_source, notes)
    VALUES (?, ?, ?, ?, ?)
''', ('monthly_analysis', len(merged_monthly), datetime.now().isoformat(),
      'Derived from air_quality and hospital_visits',
      'Monthly aggregated data for correlation and regression analysis'))
conn.commit()

Loaded 555 monthly analysis records into database


In [19]:
# database summary
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
tables = cursor.fetchall()

print("="*60)
print("DATABASE SUMMARY")
print("="*60)

for (table_name,) in tables:
    cursor.execute(f"SELECT COUNT(*) FROM {table_name}")
    count = cursor.fetchone()[0]
    print(f"{table_name:30} {count:>10,} records")

print("="*60)

DATABASE SUMMARY
sqlite_sequence                         1 records
metadata                                6 records
air_quality                        30,697 records
hospital_visits                    55,000 records
monthly_analysis                      555 records


In [20]:
# display metadata
metadata_df = pd.read_sql('SELECT * FROM metadata ORDER BY last_updated DESC', conn)
metadata_df

,id,table_name,record_count,last_updated,data_source,notes
0,6,monthly_analysis,555,2025-12-09T19:13:08.404148,Derived from air_quality and hospital_visits,Monthly aggregated data for correlation and re...
1,5,hospital_visits,55000,2025-12-09T19:13:08.166835,City of Chicago Open Data Portal,Weekly respiratory illness ED visits by catego...
2,4,air_quality,30697,2025-12-09T19:13:07.976574,EPA AQS API (dailyData/byCounty),Chicago PM2.5 daily measurements 2015-2024
3,3,monthly_analysis,555,2025-12-09T12:25:57.104142,Derived from air_quality and hospital_visits,Monthly aggregated data for correlation and re...
4,2,hospital_visits,55000,2025-12-09T12:25:56.850531,City of Chicago Open Data Portal,Weekly respiratory illness ED visits by catego...
5,1,air_quality,30697,2025-12-09T12:25:56.633873,EPA AQS API (dailyData/byCounty),Chicago PM2.5 daily measurements 2015-2024


In [21]:
# monthly analysis summary statistics
query = '''
    SELECT 
        respiratory_category,
        parameter,
        COUNT(*) as month_count,
        ROUND(AVG(avg_pollution), 2) as mean_pollution,
        ROUND(AVG(avg_visit_percent), 4) as mean_visit_pct,
        ROUND(MIN(avg_pollution), 2) as min_pollution,
        ROUND(MAX(avg_pollution), 2) as max_pollution
    FROM monthly_analysis
    GROUP BY respiratory_category, parameter
    ORDER BY respiratory_category, parameter
'''

summary = pd.read_sql(query, conn)
summary

,respiratory_category,parameter,month_count,mean_pollution,mean_visit_pct,min_pollution,max_pollution
0,Broad Acute Respiratory,PM2.5 - Local Conditions,111,8.95,0.1209,4.14,20.71
1,COVID-19,PM2.5 - Local Conditions,111,8.95,0.0137,4.14,20.71
2,ILI,PM2.5 - Local Conditions,111,8.95,0.0176,4.14,20.71
3,Influenza,PM2.5 - Local Conditions,111,8.95,0.0062,4.14,20.71
4,RSV,PM2.5 - Local Conditions,111,8.95,0.0020,4.14,20.71


In [22]:
# close database connection
conn.close()
print("Database connection closed")
print(f"Database location: {os.path.abspath(db_path)}")

Database connection closed
Database location: /Users/kirtan_patel/Documents/477/477project/data/analysis.db
